In [8]:
import sys
import os
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))

if project_root not in sys.path:
    sys.path.append(project_root)
    print(f"프로젝트 루트를 sys.path에 추가했습니다: {project_root}")

In [9]:
#from src.core.config import settings
import gspread
from langchain_openai import ChatOpenAI
from google.oauth2.service_account import Credentials
from datetime import datetime
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser


In [11]:
load_dotenv()

True

In [12]:
llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0, verbose=True)

In [13]:
SCOPES = [
    "https://www.googleapis.com/auth/drive",
]

In [14]:
def read_transcript_from_path(file_path: str):
    """ 로컬 경로에서 텍스트 파일을 읽습니다. """
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()
            return content
    except FileNotFoundError as e:
        print("해당 파일을 찾을 수 없습니다: {e}")
        return None
    except Exception as e:
        print("파일을 로드 중 오류가 발생했습니다: {e}")
        return None


In [ ]:

def parse_with_openai(text_to_parsing: str):
    """ GPT를 이용해 회의 텍스트를 요약 및 JSON 형식으로 파싱 합니다. """

    system_prompt = """
    당신은 전문 회의록을 분석하여 Google Sheets에 입력할 데이터를 추출하는 전문 어시스턴트입니다.
    회의록을 분석하여 다음 JSON 구조에 맞춰 "전역", "AI", "언리얼"의 DONE, TO DO, ISSUE 사항을 추출해야합니다.
    - 각 팀원 별로 항목을 분리해야합니다.
    - 언급된 내용이 없으면 빈 문자열("") 이나 빈 리스트([])로 채워주세요.
    - 회의 날짜를 "YYYY.MM.DD" 형식으로 추출해야 합니다.
    - AI 팀은 3명이고, 언리얼 팀은 2명인걸 명심해주세요.
    - 최대한 원본 회의록의 내용을 보존한 채로 요약 & 파싱해야 합니다.
    - task는 작업의 핵심만 담아서 최대한 간결하게 작성해야 합니다.

    반드시 다음 JSON 형식으로만 응답해야 합니다:
    {{
    "global": {{
        "일자": "회의록에 적힌 일자",
        "구분": "전역",
        "DONE": "팀원 전체가 완료한 일 요약",
        "TO DO": "팀원 전체가 해야 할 일 요약",
        "ISSUE": "팀원 전체의 이슈 사항 요약"
    }},
    "ai_team": {{
        "일자": "회의록에 적힌 일자",
        
        "구분": "AI",
        "DONE": [
        {{"member": "AI 팀원 1", "task": "완료한 작업 내용"}},
        {{"member": "AI 팀원 2", "task": "완료한 작업 내용"}},
        {{"member": "AI 팀원 3", "task": "완료한 작업 내용"}},
        ],
        "TO DO": [
        {{"member": "AI 팀원 1", "task": "해야할 작업 내용"}},
        {{"member": "AI 팀원 2", "task": "해야할 작업 내용"}},
        {{"member": "AI 팀원 3", "task": "해야할 작업 내용"}},
        ],
        "ISSUE": [
        {{"member": "AI 팀원 1", "task": "발생한 이슈 내용"}},
        {{"member": "AI 팀원 2", "task": "발생한 이슈 내용"}},
        {{"member": "AI 팀원 3", "task": "발생한 이슈 내용"}},
        ]
    }},
    "unreal_team": {{
        "일자": "회의록에 적힌 일자",
        "구분": "언리얼",
        "DONE": [
        {{"member": "언리얼 팀원 1", "task": "완료한 작업 내용"}},
        {{"member": "언리얼 팀원 2", "task": "완료한 작업 내용"}},
        ],
        "TO DO": [
        {{"member": "언리얼 팀원 1", "task": "해야할 작업 내용"}},
        {{"member": "언리얼 팀원 2", "task": "해야할 작업 내용"}},
        ],
        "ISSUE": [
        {{"member": "언리얼 팀원 1", "task": "발생한 이슈 내용"}},
        {{"member": "언리얼 팀원 2", "task": "발생한 이슈 내용"}},
        ]
    }}
    }}
    """

    chat_prompt = ChatPromptTemplate.from_messages(
        [
            ('system', system_prompt),
            ('user', '{content}')
        ]
    )

    chain = chat_prompt | llm | JsonOutputParser()
    response = chain.invoke({
        "content": text_to_parsing
    })

    print(response)
    return response


In [33]:
content = read_transcript_from_path("../data/2025_11_13_회의록.txt")
len(content)

27426

In [ ]:
response = parse_with_openai(content)
response

{'global': {'일자': '2025.11.13', '구분': '전역', 'DONE': '전체적인 구직자와 기업 간 AI 기반 가상 면접 및 모의 면접 서비스의 기본 구도와 프로세스 기획 완료. 문서화 및 API 명세서 작성 준비 중. 기본적인 회원 가입 및 로그인 기능 구현 시작.', 'TO DO': '화상 면접 및 멀티플레이 기능 구현 검토 및 개발. AI 클론 정확도 고도화 및 선호도 반영 질문지 개발. 기업별 부스 및 멀티 플레이 환경 구성. UI/UX 디자인 및 화면 구성 작업 진행. API 및 파라미터 정리, 자동화 도구 개발. 문서 뷰어 및 DRM 적용 방안 검토.', 'ISSUE': 'AI 클론의 정확도 및 다각적 질문 구성의 어려움. 문서 검증 및 인증 시스템의 복잡성. 멀티 플레이 환경에서 기업과 구직자 간 동시 접속 및 관리 문제. 캐릭터 및 얼굴 인식 로그인 구현 난이도. 기업별 맞춤 부스 및 박람회장 구성에 대한 기획 방향 불명확. 서비스 차별성 및 구매력 확보 방안 고민.'}, 'ai_team': {'일자': '2025.11.13', '구분': 'AI', 'DONE': [{'member': 'AI 팀원 1', 'task': 'AI 기반 면접 클론 생성 및 인증 로직 기획. 서류 및 기술 면접 자동화 프로세스 설계.'}, {'member': 'AI 팀원 2', 'task': 'AI 면접관 클론과 구직자 클론 간 대화 시나리오 및 매칭 로직 논의.'}, {'member': 'AI 팀원 3', 'task': 'AI 클론 정확도 향상을 위한 다각적 질문 구성 및 선호도 반영 방안 검토.'}], 'TO DO': [{'member': 'AI 팀원 1', 'task': 'AI 클론의 말투, 태도, 기술 수준 검증 고도화 작업.'}, {'member': 'AI 팀원 2', 'task': 'AI 면접관과 구직자 클론 간 멀티플레이 대화 기능 개발.'}, {'member': 'AI 팀원 3', 'task': '선호도 기반 질문지 및 평가 시스템 개

{'global': {'일자': '2025.11.13',
  '구분': '전역',
  'DONE': '전체적인 구직자와 기업 간 AI 기반 가상 면접 및 모의 면접 서비스의 기본 구도와 프로세스 기획 완료. 문서화 및 API 명세서 작성 준비 중. 기본적인 회원 가입 및 로그인 기능 구현 시작.',
  'TO DO': '화상 면접 및 멀티플레이 기능 구현 검토 및 개발. AI 클론 정확도 고도화 및 선호도 반영 질문지 개발. 기업별 부스 및 멀티 플레이 환경 구성. UI/UX 디자인 및 화면 구성 작업 진행. API 및 파라미터 정리, 자동화 도구 개발. 문서 뷰어 및 DRM 적용 방안 검토.',
  'ISSUE': 'AI 클론의 정확도 및 다각적 질문 구성의 어려움. 문서 검증 및 인증 시스템의 복잡성. 멀티 플레이 환경에서 기업과 구직자 간 동시 접속 및 관리 문제. 캐릭터 및 얼굴 인식 로그인 구현 난이도. 기업별 맞춤 부스 및 박람회장 구성에 대한 기획 방향 불명확. 서비스 차별성 및 구매력 확보 방안 고민.'},
 'ai_team': {'일자': '2025.11.13',
  '구분': 'AI',
  'DONE': [{'member': 'AI 팀원 1',
    'task': 'AI 기반 면접 클론 생성 및 인증 로직 기획. 서류 및 기술 면접 자동화 프로세스 설계.'},
   {'member': 'AI 팀원 2', 'task': 'AI 면접관 클론과 구직자 클론 간 대화 시나리오 및 매칭 로직 논의.'},
   {'member': 'AI 팀원 3',
    'task': 'AI 클론 정확도 향상을 위한 다각적 질문 구성 및 선호도 반영 방안 검토.'}],
  'TO DO': [{'member': 'AI 팀원 1', 'task': 'AI 클론의 말투, 태도, 기술 수준 검증 고도화 작업.'},
   {'member': 'AI 팀원 2', 'task': 'AI 면접관과 구직자 클론 간 멀티플레이 대화 기능 개발.'},
   {'member': 'AI 팀원

In [35]:
print(type(response))

<class 'dict'>


In [ ]:
import os, gspread
import pandas as pd
from enum import Enum
from pydantic import BaseModel, Field
from typing import Optional, Literal
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow


## 인증 처리 함수
def auth(credentials_file:str) -> Credentials:
    """Google API를 사용하기 위한 사용자 인증을 처리합니다.
    token.json 파일이 있는지 확인합니다. 있다면 유효하고, 없거나 만료되었다면
    credentials.json 을 사용해 사용자에게 브라우저를 통한 로그인을 요청합니다.
    로그인에 성공하면 새로운 인증 정보를 token.json 파일로 저장하고 
    Credentials 객체를 반환합니다."""
    creds : Credentials | None = None

    # creds: Optional[Credentials] = None
    if os.path.exists("token.json"):
        creds = Credentials.from_authorized_user_file("token.json", SCOPES)
    
    # 유효한 인증 정보가 없으면, 사용자에게 로그인을 요청
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(credentials_file, SCOPES)
            creds = flow.run_local_server(port=0)
        
        # 다음 실행을 위해 인증 정보를 저장합니다.
        with open("token.json", "w") as token:
            token.write(creds.to_json())
    
    return creds

In [39]:
g_data = response.get('global', {})
g_data.get('일자', '')

'2025.11.13'

In [40]:
ai_data = response.get('ai_team', {})
ai_data.get('DONE', '')

[{'member': 'AI 팀원 1',
  'task': 'AI 기반 면접 클론 생성 및 인증 로직 기획. 서류 및 기술 면접 자동화 프로세스 설계.'},
 {'member': 'AI 팀원 2', 'task': 'AI 면접관 클론과 구직자 클론 간 대화 시나리오 및 매칭 로직 논의.'},
 {'member': 'AI 팀원 3', 'task': 'AI 클론 정확도 향상을 위한 다각적 질문 구성 및 선호도 반영 방안 검토.'}]

In [46]:
def format_for_team_tasks(task_list: list):
    """
    [{'member': '이름', 'task': '할일'}, ...] 형태의 리스트를
    '- (이름) 할일\n- (이름) 할일' 형태의 문자열로 변환합니다.
    """
    if not task_list: return ""

    formatted_lines = []
    for item in task_list:
        member = item.get('member', '').strip()
        task = item.get('task', '').strip()

        if member or task:
            line = f"- ({member}) {task}"
            formatted_lines.append(line)

    return "\n".join(formatted_lines)        



In [47]:
print(format_for_team_tasks(ai_data.get('DONE', '')))

- (AI 팀원 1) AI 기반 면접 클론 생성 및 인증 로직 기획. 서류 및 기술 면접 자동화 프로세스 설계.
- (AI 팀원 2) AI 면접관 클론과 구직자 클론 간 대화 시나리오 및 매칭 로직 논의.
- (AI 팀원 3) AI 클론 정확도 향상을 위한 다각적 질문 구성 및 선호도 반영 방안 검토.


In [ ]:
credentials_file = "../credentials.json"
def update_daily_report_sheet(response):
    """ 회의록에서 추출된 데이터로 Google Sheets의 5, 6, 7행을 업데이트합니다. """
    try:
        creds = auth(credentials_file=credentials_file)
        client = gspread.authorize(creds)
        sheet = client.open_by_key(os.getenv("DAILY_NOTES_GOOGLE_SHEET_ID"))

        cells_to_update = []

        # 5행 (구분: 전역)
        global_data = response.get('global', {})
        cells_to_update.append(gspread.Cell(5, 1, global_data.get('일자', '')))
        cells_to_update.append(gspread.Cell(5, 2, global_data.get('구분', '전역')))
        cells_to_update.append(gspread.Cell(5, 3, global_data.get('DONE', '')))
        cells_to_update.append(gspread.Cell(5, 4, global_data.get('TO DO', '')))
        cells_to_update.append(gspread.Cell(5, 5, global_data.get('ISSUE', '')))

        # 6행 (구분: AI)
        ai_data = response.get('ai_team', {})
        cells_to_update.append(gspread.Cell(6, 1, ai_data.get('일자', '')))
        cells_to_update.append(gspread.Cell(6, 2, ai_data.get('구분', 'AI')))
        cells_to_update.append(gspread.Cell(6, 3, format_for_team_tasks(ai_data.get('DONE', ''))))
        cells_to_update.append(gspread.Cell(6, 4, format_for_team_tasks(ai_data.get('TO DO', ''))))
        cells_to_update.append(gspread.Cell(6, 5, format_for_team_tasks(ai_data.get('ISSUE', ''))))

        # 7행 (구분: 언리얼)
        ue_data = response.get('unreal_team', {})
        cells_to_update.append(gspread.Cell(6, 1, ue_data.get('일자', '')))
        cells_to_update.append(gspread.Cell(6, 2, ue_data.get('구분', '언리얼')))
        cells_to_update.append(gspread.Cell(6, 3, format_for_team_tasks(ue_data.get('DONE', ''))))
        cells_to_update.append(gspread.Cell(6, 4, format_for_team_tasks(ue_data.get('TO DO', ''))))
        cells_to_update.append(gspread.Cell(6, 5, format_for_team_tasks(ue_data.get('ISSUE', ''))))

    except Exception as e:
        print(f"Goolge Sheet 없데이트 실패: {e}")




TypeError: the JSON object must be str, bytes or bytearray, not dict

In [28]:
print(type(response))

<class 'NoneType'>


In [ ]:
def write_to_google_sheet(meeting_title, summary_text):
    """ Google Sheets에 최종 결과를 기록합니다."""